In [129]:
# use to reload edited file
%load_ext autoreload

%autoreload 2

# Preprocess prophet dataset
## v3.2
### preprocess raw dataset

In [27]:
from pathlib import Path
import pickle

path_news_filter_df = Path('results/news_filter_df_v3.2.pkl')
with path_news_filter_df.open('rb') as f:
    df1 = pickle.load(f)
df1.head()

,esid,prefix,input_text,target_text
0,52e10537cd0eb295f23391d5274b9002,news_filter,全球投融资周报(2018.8.31-9.6)丨 百度视频宣布完成1亿美元B轮融资;大搜车获5...,医药
1,5dc35789ed5eb6c700a735ff4e600097,news_filter,"医疗机构领健完成数千万 B+轮融资,美团点评领投 | IT桔子周报 编辑：王炜本周共计发生 ...",医药
2,c698937f8670b0e6763dd929c1fdb6ec,news_filter,9月10日临创投资晨报 以下市场资讯由临创投资研究所整理摘要：1、宏观经济与政策：中国8月出...,医药
3,684faad1f4de34127cf1101d8767a332,news_filter,"民祥医药完成C轮融资,估值超10亿元 兴富资本投资的新三板公司民祥医药(833004)喜获C...",医药
4,999c03bc10f03d7f88b87806f213d0b,news_filter,线闻|下一代肿瘤免疫新锐药企“寻百会生物”获数千万元新一轮融资 小线菌2018年10月1日，...,医药


In [31]:
df1.columns

Index(['esid', 'prefix', 'input_text', 'target_text'], dtype='object')

In [32]:
!rm results/news_filter_df_v3.2.pkl


### export to-check dataset

In [9]:
import pandas as pd
from pharm_ai.prophet.dt import ProphetT5PreprocessorV3_1, ProphetT5PreprocessorV3_2

In [31]:
prepro = ProphetT5PreprocessorV3_2()
raw_df = prepro.preprocess_raw_not_check()


2021-07-12 17:13:32.591 | INFO     | pharm_ai.prophet.dt:get_company_dictionary:361 - 25483 company names loaded from "results/company_dictionary.json".
2021-07-12 17:13:32.658 | INFO     | pharm_ai.prophet.dt:get_invester_list:376 - 31298 investers loaded from "results/invester_list.json".
2021-07-12 17:13:35.272 | INFO     | pharm_ai.prophet.dt:preprocess_raw:206 - News_filter dataset loaded from "results/news_filter_df_v3.2.pkl".
2021-07-12 17:13:35.356 | INFO     | pharm_ai.prophet.dt:preprocess_raw:223 - IRA&org dataset loaded from "results/ira_org_df_v3.2.pkl".


In [73]:
Path('results/news_filter_data_v3.2.json').exists()


True

In [131]:
last_prepro = ProphetT5PreprocessorV3_1()
raw_last = last_prepro.preprocess_raw(fill_empty_paragraph=False,
                                      ignore_org_ner=True)

2021-07-13 14:30:42.231 | INFO     | pharm_ai.prophet.ira.prepro:get_from_h5:57 - Read data from '/home/fzq/miniconda3/envs/ai/lib/python3.9/site-packages/pharm_ai/prophet/ira/data.h5' with key raw/v2-5


In [132]:
raw_last['prefix'].unique()

array(['news_filter', 'investee', 'round', 'amount', 'org_filter'],
      dtype=object)

In [32]:
raw_df['prefix'].unique()


array(['news_filter', 'ira_filter', 'org_filter', 'investee', 'round',
       'amount', 'org_ner.c3', 'org_ner.c1', 'org_ner', 'org_ner.c2'],
      dtype=object)

#### ira_filter data

Last version ira_filter data lost.

In [232]:
ira_filter_df = raw_df[raw_df['prefix']=='ira_filter']


In [236]:
ira_filter_df.head()

,esid,prefix,titles,input_text,target_text,publish_date
570818,afd969d102dc4b589b674a35b9d95fec,ira_filter,NaN,"NEW YORK, Feb. 9, 2021 /PRNewswire/ -- Cambria...",1,2021-02-08 16:00:00
570822,afd969d102dc4b589b674a35b9d95fec,ira_filter,NaN,This launch coincides with the IPO of its port...,0,2021-02-08 16:00:00
570824,afd969d102dc4b589b674a35b9d95fec,ira_filter,NaN,Cambrian Biopharma announces $60M in funding t...,0,2021-02-08 16:00:00
570826,afd969d102dc4b589b674a35b9d95fec,ira_filter,NaN,"Cambrian scientists are targeting the ""Nine Ha...",0,2021-02-08 16:00:00
570828,afd969d102dc4b589b674a35b9d95fec,ira_filter,NaN,"""Over the next decade, Cambrian aims to detect...",0,2021-02-08 16:00:00


In [238]:
ira_filter_df.isna().sum()


esid                0
prefix              0
titles          18616
input_text          0
target_text         0
publish_date        0
dtype: int64

In [245]:
ira_filter_to_check = ira_filter_df[['esid', 'publish_date', 'input_text', 'target_text']].rename(
    columns={'input_text': 'paragraphs', 'target_text': 'predict_label'}
)
ira_filter_to_check['predict_label'] = ira_filter_to_check['predict_label'].astype(int)
ira_filter_to_check['true_label'] = None
ira_filter_to_check.head()

,esid,publish_date,paragraphs,predict_label,true_label
570818,afd969d102dc4b589b674a35b9d95fec,2021-02-08 16:00:00,"NEW YORK, Feb. 9, 2021 /PRNewswire/ -- Cambria...",1,None
570822,afd969d102dc4b589b674a35b9d95fec,2021-02-08 16:00:00,This launch coincides with the IPO of its port...,0,None
570824,afd969d102dc4b589b674a35b9d95fec,2021-02-08 16:00:00,Cambrian Biopharma announces $60M in funding t...,0,None
570826,afd969d102dc4b589b674a35b9d95fec,2021-02-08 16:00:00,"Cambrian scientists are targeting the ""Nine Ha...",0,None
570828,afd969d102dc4b589b674a35b9d95fec,2021-02-08 16:00:00,"""Over the next decade, Cambrian aims to detect...",0,None


#### ira

In [9]:
ira_df = raw_df[raw_df['prefix'].isin(['investee', 'round', 'amount'])]
ira_df.head()

,esid,prefix,titles,input_text,target_text
285410,afd969d102dc4b589b674a35b9d95fec,round,NaN,"NEW YORK, Feb. 9, 2021 /PRNewswire/ -- Cambria...",战略投资
285411,afd969d102dc4b589b674a35b9d95fec,amount,NaN,"NEW YORK, Feb. 9, 2021 /PRNewswire/ -- Cambria...",6000万美元
285424,563f67d11f5c46ae97072be238aa79fd,round,NaN,"NEW YORK, Feb. 10, 2021 /PRNewswire/ -- Scopus...",增发
285425,563f67d11f5c46ae97072be238aa79fd,amount,NaN,"NEW YORK, Feb. 10, 2021 /PRNewswire/ -- Scopus...",1035万美元
285439,81bcc86de578c89d33448c2650c10e0c,round,NaN,Aptar Acquires Nanopharm and Gateway Analytica...,并购


In [10]:
ira_df.shape

(19792, 5)

In [68]:
ira_last_df = raw_last[raw_last['prefix'].isin(['investee', 'round', 'amount'])]
ira_last_df.head()

,titles,target_text,input_text,esid,prefix
59232,NaN,Chondrial Therapeutics,"December 18, 2019 06:00 ET | Source: Zafgen, I...",NaN,investee
59233,NaN,Myst Therapeutics,"Turnstone Biologics Corp., a clinical-stage bi...",NaN,investee
59234,NaN,Medical Data Analytics,"RESEARCH TRIANGLE PARK, N.C., April 6, 2020 /P...",NaN,investee
59235,NaN,The Burgess Group,HealthEdge Software Acquires The Burgess Group...,NaN,investee
59236,NaN,Amryt Pharma,据6月23日美国证监会文件披露，英国上市的生物制药公司Amryt Pharma将以“AMYT...,NaN,investee


In [17]:
ira_last_df.shape

(27021, 5)

In [18]:
ira_last_df.isna().sum()

titles         27021
target_text        0
input_text         0
esid           27021
prefix             0
dtype: int64

Cannot use esid or columns to match, try paragraph.

In [20]:
ira_last_df['input_text'].isin(ira_df['input_text']).sum()

3606

In [78]:
filter_duplicated_target_text = lambda t5_df: pd.concat(dt
for _, dt in t5_df[t5_df[['prefix', 'input_text']].duplicated()].groupby(
    ['prefix', 'input_text']) if dt['target_text'].nunique()>1)
ira_df_duplicated = filter_duplicated_target_text(ira_df)
ira_df_duplicated.head(8)

,esid,prefix,titles,input_text,target_text
305093,9eb7e753eee84e8bbd31e570f7d8e80a,amount,NaN,36氪获悉，在线心理服务平台「壹点灵」于近日完成了由比邻星领投、约印资本跟投的数千万元A轮融...,1000万元人民币
331710,18ae64dc7df94770b39e2666e7494afe,amount,NaN,36氪获悉，在线心理服务平台「壹点灵」于近日完成了由比邻星领投、约印资本跟投的数千万元A轮融...,数千万元人民币
287008,04da344815554403af5f8cfe474a283f,amount,NaN,3i Group,未披露
287402,741a5f8f70074e31a4ac3af55586c5c0,amount,NaN,3i Group,700万欧元
304079,55fd860f5dea40e3beb53a98f566a94a,amount,NaN,3i Group,650万美元
303948,5febd847c6eb4b708e0447179d5fe9ff,amount,NaN,ATDC,130.4901万美元
304044,52c0820e1c5c45b1b8d7fb9cc09c8540,amount,NaN,ATDC,250万美元
304118,3218195e66284b2880a20902053c5f39,amount,NaN,ATDC,未披露


In [76]:
ira_df_unique = ira_df.drop_duplicates(subset=['prefix', 'input_text'], keep='last')
ira_df_unique.shape

(19574, 5)

In [66]:
ira_last_duplicated = filter_duplicated_target_text(ira_last_df)
ira_last_duplicated

,titles,target_text,input_text,esid,prefix
81678,NaN,500万元人民币,36氪获悉，在线心理服务平台「壹点灵」于近日完成了由比邻星领投、约印资本跟投的数千万元A轮融...,NaN,amount
85401,NaN,1000万元人民币,36氪获悉，在线心理服务平台「壹点灵」于近日完成了由比邻星领投、约印资本跟投的数千万元A轮融...,NaN,amount
79868,NaN,数亿元人民币,1月3日消息，远洋集团旗下远洋资本完成对美中宜和医疗数亿元人民币战略投资。这是继布局养老...,NaN,amount
84703,NaN,1亿元人民币,1月3日消息，远洋集团旗下远洋资本完成对美中宜和医疗数亿元人民币战略投资。这是继布局养老...,NaN,amount
61239,NaN,钥世圈云健康,36氪获悉，上海昕健医疗技术有限公司已于2018年初完成数千万A轮融资，本轮融资由比邻星创投...,NaN,investee
61418,NaN,昕健医疗,36氪获悉，上海昕健医疗技术有限公司已于2018年初完成数千万A轮融资，本轮融资由比邻星创投...,NaN,investee
60912,NaN,天元药业集团,7月9日 - 延安必康（002411）晚间公告称，公司全资子公司陕西必康制药集团控股有限公司...,NaN,investee
60933,NaN,昌鹤医药,7月9日 - 延安必康（002411）晚间公告称，公司全资子公司陕西必康制药集团控股有限公司...,NaN,investee
61019,NaN,晋欣医药,7月9日 - 延安必康（002411）晚间公告称，公司全资子公司陕西必康制药集团控股有限公司...,NaN,investee
70246,NaN,天使轮,36氪获悉，上海昕健医疗技术有限公司已于2018年初完成数千万A轮融资，本轮融资由比邻星创投...,NaN,round


In [69]:
ira_last_df_unique=ira_last_df.drop_duplicates(subset=['prefix', 'input_text'], keep='last')
ira_last_df_unique.shape

(26880, 5)

In [109]:
ira_merged = ira_df_unique.merge(ira_last_df_unique, on=['prefix', 'input_text'], validate='1:1')
ira_not_check = ira_merged[ira_merged['target_text_x'].eq(ira_merged['target_text_y'])][
    ['esid_x', 'prefix', 'input_text', 'target_text_x']].rename(
    columns={'esid_x': 'esid','target_text_x': 'target_text'})
ira_to_check_1 = ira_merged[ira_merged['target_text_x'].ne(ira_merged['target_text_y'])][
    ['esid_x', 'prefix', 'input_text', 'target_text_y']].rename(
    columns={'esid_x': 'esid','target_text_y': 'target_text'})
ira_to_check_1[['input_text', 'target_text']].head(20)

,input_text,target_text
256,百济神州（纳斯达克代码：BGNE）今日宣布新一轮8亿美元公开募股完成。作为一家专注于开发和商...,战略投资
332,对于流通环节的经销商来说，不论是线下还是线上模式，赚的都是信息不对称的钱。而邹瑾想做的是把帘...,A轮
799,2019年5月20日，Inova Personalized Health Accelerat...,230万欧元
821,上海合璞医疗科技有限公司（以下简称“上海合璞医疗”）为了业务的拓展拟进行增资扩股，注册资本将...,1.434亿元人民币
890,12月7日，互联网体检服务平台医通健康宣布完成近2亿元人民币的B轮融资。本次B轮融资是业内第...,B轮
891,12月7日，互联网体检服务平台医通健康宣布完成近2亿元人民币的B轮融资。本次B轮融资是业内第...,近2亿元人民币
978,日化清洁用品的包装上常常会有“抗菌、抑菌”等宣传标签，然而这些抗菌成分是否有杀菌效果，这一点...,天使轮
979,日化清洁用品的包装上常常会有“抗菌、抑菌”等宣传标签，然而这些抗菌成分是否有杀菌效果，这一点...,500万元人民币
1038,36氪获悉，定位于医院病床旁场景下信息、娱乐、消费三位一体的综合服务平台「爱汇健康」宣布已于...,天使轮
1039,36氪获悉，定位于医院病床旁场景下信息、娱乐、消费三位一体的综合服务平台「爱汇健康」宣布已于...,1200万元人民币


In [110]:
ira_merged_full = ira_df_unique.merge(ira_last_df_unique, how='outer', on=['prefix', 'input_text'],
                                      validate='1:1', indicator=True)
ira_to_check_2 = ira_merged_full[ira_merged_full['_merge'].ne('both')]
ira_to_check_2['target_text'] = ira_to_check_2['target_text_x'].where(
    ira_to_check_2['target_text_x'].notna(), ira_to_check_2['target_text_y']
)
ira_to_check_2 = ira_to_check_2[['esid_x', 'prefix', 'input_text', 'target_text']].rename(
    columns={'esid_x': 'esid'}
)
ira_to_check = pd.concat([ira_to_check_1, ira_to_check_2])
ira_to_check.describe()

<ipython-input-110-df7c41c801f6>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ira_to_check_2['target_text'] = ira_to_check_2['target_text_x'].where(


,esid,prefix,input_text,target_text
count,17226,41740,41740,41740
unique,8393,3,17564,8359
top,30faf198854f4299b7ca65a9c0bd459,round,公司5.17晚发布公告，拟2728万美元现金收购BIOMEDICAL RESEARCH ...,未披露
freq,30,16391,3,4981


#### org_filter data


In [133]:
org_filter_df = raw_df[raw_df['prefix']=='org_filter']
org_filter_df.describe()

<ipython-input-133-dfb678342707>:2: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  org_filter_df.describe()


,esid,prefix,titles,input_text,target_text,publish_date
count,18616,18616,0,18616,18616,18616
unique,1387,1,0,16372,2,396
top,e3bd3dcd34ca4d68b8c3f6814ec67c88,org_filter,NaN,,0,2021-01-06 16:00:00
freq,276,18616,NaN,819,17725,701
first,NaN,NaN,NaN,NaN,NaN,2006-04-06 16:00:00
last,NaN,NaN,NaN,NaN,NaN,2021-03-18 16:00:00


In [133]:
org_filter_last = raw_last[raw_last['prefix']=='org_filter']
org_filter_last.describe()

,titles,target_text,input_text,esid,prefix
count,0,14855,14855,0,14855
unique,0,2,10355,0,1
top,NaN,0,本文发布日期为2020年9月14日。【本文为投资界原创，网页转载须在文首注明来源投资界（微信...,NaN,org_filter
freq,NaN,13735,24,NaN,14855


In [134]:
pattern = r'(?:本文发布日期为(?P<date_cn>\d+年\d+月\d+日)。(?P<para_cn>.*))|(?:This article was published on (?P<date_en>\d+-\d+-\d+). (?P<para_en>.*))'
org_filter_last['input_text'].str.extract(pattern).isna().sum()

date_cn     4143
para_cn     4143
date_en    10712
para_en    10712
dtype: int64

In [135]:
org_filter_last_ex = org_filter_last['input_text'].str.extract(pattern)
org_filter_last_ex['date_cn']=pd.to_datetime(org_filter_last_ex['date_cn'], format="%Y年%m月%d日")
org_filter_last_ex['date_en']=pd.to_datetime(org_filter_last_ex['date_en'], format="%Y-%m-%d")
org_filter_last['input_text'] = org_filter_last_ex['para_cn'].where(
    org_filter_last_ex['para_cn'].notna(), org_filter_last_ex['para_en'])
org_filter_last['publish_date'] = org_filter_last_ex['date_cn'].where(
    org_filter_last_ex['date_cn'].notna(), org_filter_last_ex['date_en'])

<ipython-input-135-75897246989e>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  org_filter_last['input_text'] = org_filter_last_ex['para_cn'].where(
<ipython-input-135-75897246989e>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  org_filter_last['publish_date'] = org_filter_last_ex['date_cn'].where(


In [136]:
org_filter_to_check = org_filter_df
org_filter_not_check = org_filter_last

In [241]:
org_filter_to_check.isna().sum()

esid                0
prefix              0
titles          18616
input_text          0
target_text         0
publish_date        0
dtype: int64

In [243]:
org_filter_to_check_df = org_filter_to_check[['esid', 'publish_date', 'input_text', 'target_text']].rename(
    columns={'input_text': 'paragraphs', 'target_text': 'predicted_label'}
)
org_filter_to_check_df['predicted_label'] = org_filter_to_check_df['predicted_label'].astype(int)
org_filter_to_check_df['true_label'] = None
org_filter_to_check_df.head()

,esid,publish_date,paragraphs,predicted_label,true_label
570819,afd969d102dc4b589b674a35b9d95fec,2021-02-08 16:00:00,"NEW YORK, Feb. 9, 2021 /PRNewswire/ -- Cambria...",0,None
570823,afd969d102dc4b589b674a35b9d95fec,2021-02-08 16:00:00,This launch coincides with the IPO of its port...,0,None
570825,afd969d102dc4b589b674a35b9d95fec,2021-02-08 16:00:00,Cambrian Biopharma announces $60M in funding t...,0,None
570827,afd969d102dc4b589b674a35b9d95fec,2021-02-08 16:00:00,"Cambrian scientists are targeting the ""Nine Ha...",0,None
570829,afd969d102dc4b589b674a35b9d95fec,2021-02-08 16:00:00,"""Over the next decade, Cambrian aims to detect...",0,None


#### org_ner

In [216]:
org_ner_df = raw_df[raw_df['prefix'].str.startswith('org_ner')]

In [218]:
org_ner_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9699 entries, 570876 to 637540
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   esid          9699 non-null   object        
 1   prefix        9699 non-null   object        
 2   titles        0 non-null      object        
 3   input_text    9699 non-null   object        
 4   target_text   9699 non-null   object        
 5   publish_date  0 non-null      datetime64[ns]
dtypes: datetime64[ns](1), object(5)
memory usage: 530.4+ KB


In [220]:
org_ner_df['publish_date'].isnull().sum()

9699

In [228]:
org_ner_ex = org_ner_df['prefix'].str.split('.', expand=True)

In [229]:
org_ner_df['prefix'] = org_ner_ex.loc[:,0]
org_ner_df['org_type'] = org_ner_ex.loc[:,1]

<ipython-input-229-c48472a6773c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  org_ner_df['prefix'] = org_ner_ex.loc[:,0]
<ipython-input-229-c48472a6773c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  org_ner_df['org_type'] = org_ner_ex.loc[:,1]


In [231]:
org_ner_to_check = org_ner_df[['prefix', 'input_text', 'org_type', 'target_text']]

#### write to-check data

In [244]:
with pd.ExcelWriter('results/prophet_tocheck_v3_20210706.xlsx') as writer:
    ira_filter_to_check.to_excel(writer, sheet_name='ira_filter', index=False)
    org_filter_to_check_df.to_excel(writer, sheet_name='org_filter', index=False)

In [271]:
ira_filter_to_check.to_csv('results/prophet_tocheck_ira_v3_20210707.csv')

### Update checked data

In [85]:
checked_xlsx = "raw_data/prophet_tocheck_v3_20210706已核对.xlsx"
with pd.ExcelFile(checked_xlsx) as f:
    checked_ira_filter = pd.read_excel(f, sheet_name='ira_filter',
                                       dtype={'predict_label': 'str', 'true_label': 'str'})
    checked_org_filter = pd.read_excel(f, sheet_name='org_filter',
                                       dtype={'predict_label': 'str', 'true_label': 'str'})

In [86]:
column_mapper = {'true_label': 'target_text'}
checked_ira_filter=checked_ira_filter.rename(columns=column_mapper)
checked_ira_filter['prefix'] = 'ira_filter'
checked_ira_filter['publish_date'] = checked_ira_filter['publish_date'].dt.tz_localize('utc').dt.tz_convert('Asia/Shanghai')
checked_org_filter=checked_org_filter.rename(columns=column_mapper)
checked_org_filter['prefix'] = 'org_filter'
checked_org_filter['publish_date'] = checked_org_filter['publish_date'].dt.tz_localize('utc').dt.tz_convert('Asia/Shanghai')

In [91]:
import langid
def add_date_to_paragraph(date_, para):
    lang_=langid.classify(para)[0]
    if lang_=='zh':
        res_para=f'本文发布日期为{date_.year}年{date_.month}月{date_.day}日。{para}'
    else:
        res_para=f'This article was published on {date_.strftime("%B, %d, %Y")}. {para}'
    return res_para


test_entry = checked_ira_filter.iloc[2,:]
test_res_para = add_date_to_paragraph(test_entry['publish_date'], test_entry['paragraphs'])
test_res_para

'This article was published on February, 09, 2021. Cambrian Biopharma announces $60M in funding to develop pipeline of companies, therapeutics to treat diseases of aging led-with-a-unique-operational-strategy-301224991.html"> Tweet this Cambrian is embracing the idea of a "DisCo," which works like a hub-and-spoke model to develop a family of companies, allowing them to thrive by building expert teams in drug discovery, development, clinical trials, finance, and market analysis as a shared resource for each pipeline company to use.  '

In [92]:
checked_ira_filter['input_text'] = checked_ira_filter.apply(
    lambda row: add_date_to_paragraph(row['publish_date'], row['paragraphs']),
    axis=1
)

In [94]:
checked_org_filter['input_text'] = checked_org_filter.apply(
    lambda row: add_date_to_paragraph(row['publish_date'], row['paragraphs']),
    axis=1
)

In [95]:
checked_ira_filter.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18616 entries, 0 to 18615
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype                        
---  ------         --------------  -----                        
 0   esid           18616 non-null  object                       
 1   publish_date   18616 non-null  datetime64[ns, Asia/Shanghai]
 2   paragraphs     18616 non-null  object                       
 3   predict_label  18616 non-null  object                       
 4   target_text    18616 non-null  object                       
 5   prefix         18616 non-null  object                       
 6   input_text     18616 non-null  object                       
dtypes: datetime64[ns, Asia/Shanghai](1), object(6)
memory usage: 1018.2+ KB


In [137]:
not_check_t5_data = raw_df[raw_df['prefix'].isin([
    'news_filter', 'investee', 'round', 'amount'])|raw_df['prefix'].str.startswith('org_ner.')]
total_raw_data = pd.concat([not_check_t5_data, checked_ira_filter, checked_org_filter, org_filter_not_check])

In [150]:
total_raw_data = total_raw_data[
    total_raw_data['prefix'].ne('news_filter')|total_raw_data['target_text'].isin(['医药','非医药','非相关'])
]

In [151]:
total_raw_data['prefix'].value_counts()

news_filter    276826
org_filter      33471
ira_filter      18616
amount           9896
round            9896
investee         9896
org_ner.c1       3219
org_ner.c3       3165
org_ner.c2       2161
Name: prefix, dtype: int64

### train and eval dataset (upsampling)

In [152]:
from sklearn.model_selection import train_test_split
train_df, eval_df = train_test_split(total_raw_data[['esid', 'prefix', 'input_text', 'target_text']],
                                     test_size=0.1, random_state=713)

In [153]:
((total_raw_data['prefix']=='news_filter')&(~total_raw_data['target_text'].isin(['医药','非医药','非相关']))).sum()

0

In [154]:
eval_df['prefix'].value_counts()

news_filter    27651
org_filter      3360
ira_filter      1830
round           1037
amount          1002
investee         976
org_ner.c3       327
org_ner.c1       306
org_ner.c2       226
Name: prefix, dtype: int64

In [155]:
news_filter_df = train_df[train_df['prefix']=='news_filter']
ira_df = train_df[train_df['prefix'].isin(['investee', 'round', 'amount'])]
ira_filter_df=train_df[train_df['prefix']=='ira_filter']
org_filter_df=train_df[train_df['prefix']=='org_filter']
org_ner_df = train_df[train_df['prefix'].str.startswith('org_ner')]

In [157]:
news_filter_df['target_text'].value_counts()

非相关    195689
医药      33716
非医药     19770
Name: target_text, dtype: int64

In [107]:
from sklearn.utils import resample

def upsampling_t5_classification_dataset(df):
    max_label_size = df['target_text'].value_counts().max()
    res_df = pd.concat(
        resample(d, n_samples=max_label_size, random_state=123) if d.shape[0]<max_label_size else d
        for label, d in df.groupby('target_text')
    )
    return res_df

In [143]:
news_filter_upsampled = upsampling_t5_classification_dataset(news_filter_df)
ira_filter_upsampled = upsampling_t5_classification_dataset(ira_filter_df)
org_filter_upsampled = upsampling_t5_classification_dataset(org_filter_df)

train_cls_upsampled = pd.concat([news_filter_upsampled, ira_filter_upsampled,
                                     ira_df, org_filter_upsampled, org_filter_df])

In [144]:
train_cls_upsampled.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 731116 entries, 7848 to 12684
Data columns (total 4 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   esid         690946 non-null  object
 1   prefix       731116 non-null  object
 2   input_text   731116 non-null  object
 3   target_text  731116 non-null  object
dtypes: object(4)
memory usage: 27.9+ MB


In [120]:
def balance_t5_prefix(df):
    max_size = df['prefix'].value_counts().max()
    res_df = pd.concat(
        resample(d, n_samples=max_size, random_state=123) if d.shape[0]<max_size else d
        for prefix, d in df.groupby('prefix')
    )
    res_df['prefix'].value_counts()
    return res_df

In [145]:
train_upsampled = balance_t5_prefix(train_cls_upsampled)
train_upsampled['prefix'].value_counts()

round          587493
org_filter     587493
news_filter    587493
ira_filter     587493
investee       587493
amount         587493
Name: prefix, dtype: int64